下面是在http://58.241.217.181:15111/notebooks/work/ml/gnt-a/notebooks/code/customized-model-A.ipynb#
的基础上，参照：
https://www.tensorflow.org/versions/master/tutorials/layers
来制作CNN，用于识别GNT.



In [12]:
import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

IMAGE_SIZE = 64
# CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 64, 64, 1])
  print ('shape of input_layer: ', input_layer)
  # with batch_size =100, shape should be: [100, 28, 28, 1]
  # shape of input_layer:  Tensor("Reshape:0", shape=(100, 64, 64, 1), dtype=float32)

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  print ('shape of conv1: ', conv1)
  # shape of conv1:  Tensor("conv2d/Relu:0", shape=(100, 64, 64, 32), dtype=float32)
    
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  print ('shape of conv2: ', conv2, '; and shape of pool2 is: ', pool2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=CHARSET_SIZE)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


class DataSetLoader:
    def __init__(self, data_dir):
        # Set FLAGS.charset_size to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        print ('self.images: ', self.images[0].dtype)
        self.images = np.float32(self.images)
        print ('self.images: ', self.images[0].dtype)

        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

    
train_data = DataSetLoader(data_dir='../data/train_/')
print ('Train data loaded ...')


# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]



# Build 2 hidden layer DNN with ...
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,
   model_dir='../dfs/checkpoint/customized_model-c')

    

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(train_data)[0]},
    y=input(train_data)[1],
    num_epochs=None,
    batch_size=100,
    shuffle=True
)
print ('Begin to train ...')

classifier.train(input_fn=train_input_fn, steps=1000)
print ('Train done ...')

test_data = DataSetLoader(data_dir='../data/test_/')
print ('Test data loaded ...')

# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

Now processing path:  ../data/train_/00037
self.images:  float64
self.images:  float32
Train data loaded ...
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../dfs/checkpoint/customized_model-c', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
Begin to train ...
shape of input_layer:  Tensor("Reshape:0", shape=(100, 64, 64, 1), dtype=float32)
shape of conv1:  Tensor("conv2d/Relu:0", shape=(100, 64, 64, 32), dtype=float32)
shape of conv2:  Tensor("conv2d_2/Relu:0", shape=(100, 32, 32, 64), dtype=float32) ; and shape of pool2 is:  Tensor("max_pooling2d_2/MaxPool:0", shape=(100, 16, 16, 64), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ../dfs/checkpoint/customized_model-c/model.ckpt.
INFO:tensorflow:loss = 3.

自定义CNN雏形完成：
INFO:tensorflow:Starting evaluation at 2018-04-26-06:35:15
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-04-26-06:35:21
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.106335, global_step = 1000, loss = 3.58817

Test Accuracy: 10.633484%


